*Carga de Librerias*

In [4]:
import cv2
import json
from ultralytics import YOLO

def detectar_producto():
    # 1. Cargar inventario
    with open("/home/amewir/ProyectoIA/data/inventario.json", "r") as file:
        inventario = json.load(file)

    # Crear un set con los nombres de productos del inventario
    nombres_inventario = {item["name"].lower(): item for item in inventario}

    # 2. Cargar modelo YOLO
    model = YOLO("yolov8n.pt")

    # 3. Inicializar cámara
    cap = cv2.VideoCapture(0)

    print("Iniciando cámara... Presiona 'q' para salir manualmente.")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        results = model(frame)

        # Dibujar anotaciones en pantalla
        anotaciones = results[0].plot()
        cv2.imshow("Detección", anotaciones)

        # 4. Extraer detecciones
        detecciones = results[0].boxes

        if len(detecciones) > 0:
            # Tomar la primera detección
            clase_id = int(detecciones[0].cls[0])
            nombre_detectado = model.names[clase_id].lower()

            print(f"Objeto detectado: {nombre_detectado}")

            # 5. Verificar si está en el inventario
            if nombre_detectado in nombres_inventario:
                print("Producto encontrado en inventario.")
                producto = nombres_inventario[nombre_detectado]

                # Cerrar cámara y ventanas
                cap.release()
                cv2.destroyAllWindows()

                return producto  # ← devuelve todo el objeto

        # Salida manual
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cerrar cámara si se sale sin coincidencias
    cap.release()
    cv2.destroyAllWindows()
    return None
    

In [5]:
import json
from datetime import datetime


def registrar_movimiento(product_id, cantidad, tipo_movimiento):
    inventario_path = "/home/amewir/ProyectoIA/data/inventario.json"
    historial_path = "/home/amewir/ProyectoIA/data/historial.json"

    # 1. Cargar inventario
    with open(inventario_path, "r") as file:
        inventario = json.load(file)

    # Buscar producto
    producto = next((p for p in inventario if p["product_id"] == product_id), None)

    if producto is None:
        return {"error": "Producto no encontrado"}

    # 2. Actualizar stock
    if tipo_movimiento.lower() == "entrada":
        producto['stock'] += cantidad

    elif tipo_movimiento.lower() == "salida":
        if producto['stock'] < cantidad:
            return {"error": "Stock insuficiente para salida"}
        producto["stock"] -= cantidad

    else:
        return {"error": "Tipo de movimiento inválido. Use 'Entrada' o 'Salida'"}

    # Guardar nuevo inventario
    with open(inventario_path, "w") as file:
        json.dump(inventario, file, indent=4)

    # 3. Guardar movimiento en historial
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    hora_actual = datetime.now().strftime("%H:%M")

    # Cargar historial
    try:
        with open(historial_path, "r") as file:
            historial = json.load(file)
    except:
        historial = []

    # Generar ID consecutivo
    nuevo_id = historial[-1]["id"] + 1 if historial else 1

    nuevo_registro = {
        "id": nuevo_id,
        "date": fecha_actual,
        "time": hora_actual,

        "product_id": producto["product_id"],
        "product_name": producto["name"],
        "minimum_stock": producto["minimum_stock"],
        "category": producto["category"],
        "price_per_unit": producto["price"],

        "quantity": cantidad,
        "stock_after": producto["stock"],
        "movement_type": "Ingreso" if tipo_movimiento.lower() == "entrada" else "Salida"
    }

    historial.append(nuevo_registro)

    with open(historial_path, "w") as file:
        json.dump(historial, file, indent=4)

    return nuevo_registro

In [6]:
import json
import pandas as pd

def obtener_historial():
    historial_path = "/home/amewir/ProyectoIA/data/historial.json"

    # Leer historial
    with open(historial_path, "r") as file:
        datos = json.load(file)

    # Convertir a DataFrame
    df = pd.DataFrame(datos)

    return df

In [49]:


# Flujo principal

#################################################
# FUNCIÓN 1
# Detectar producto usando la cámara
#################################################
producto = detectar_producto()

# Si se detecta un producto, registrar movimiento y actualizar inventario
if producto:
    print("Producto detectado:", producto)
    
    # Menu para pruebas en consola    
    print("""Ingresa el numero del tipo de movimiento \n1. Entrada  2. Salida""")
    # La funcion recibe "Entrada" o "Salida", no el numero
    tipo_movimiento = "Entrada" if input() == "1" else "Salida"
    print("Ingresa la cantidad a registrar:")
    cantidad = int(input())
    
    ##############################################
    # FUNCIÓN 2
    # Registrar movimiento y actualizar inventario. recibe product_id, cantidad, tipo_movimiento ################################################
    resultado = registrar_movimiento(product_id=producto["product_id"], cantidad=cantidad, tipo_movimiento=tipo_movimiento)
    print(resultado)
    
# Final del flujo principal


########################################################
# FUNCION 3
# Obtener dataframe del historial de movimientos
########################################################


df_historial = obtener_historial()
print(df_historial)

Iniciando cámara... Presiona 'q' para salir manualmente.

0: 480x640 1 person, 10.4ms
Speed: 3.4ms preprocess, 10.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Objeto detectado: person

0: 480x640 1 person, 13.0ms
Speed: 2.5ms preprocess, 13.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
Objeto detectado: person

0: 480x640 1 person, 9.9ms
Speed: 2.6ms preprocess, 9.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Objeto detectado: person

0: 480x640 2 persons, 9.8ms
Speed: 1.4ms preprocess, 9.8ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Objeto detectado: person

0: 480x640 1 person, 9.8ms
Speed: 1.6ms preprocess, 9.8ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Objeto detectado: person

0: 480x640 2 persons, 9.9ms
Speed: 2.0ms preprocess, 9.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
Objeto detectado: person

0: 480x640 1 person, 9.9ms
Speed: 1.7ms pr

In [50]:
df_historial.head()

,id,date,time,product_id,product_name,minimum_stock,category,price_per_unit,quantity,stock_after,movement_type
0,1,2023-10-01,10:00,1,cell phone,5,Electronics,1000.0,2,8,Ingreso
1,2,2025-12-10,15:23,2,apple,10,fruits,5.0,50,65,Ingreso
2,3,2025-12-10,15:34,2,apple,10,fruits,5.0,10,55,Salida
3,4,2025-12-10,16:10,2,apple,10,fruits,5.0,6,61,Ingreso
4,5,2025-12-10,16:11,2,apple,10,fruits,5.0,9,70,Ingreso


In [51]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np

In [ ]:
#Estandarizamos los datos a un dataframe con labels ya definidos.

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
data = {
    'Fecha_Hora'        : pd.to_datetime(df_historial['date'] + ' ' + df_historial['time']),
    'Producto_Tienda'   : df_historial['product_name'],
    'Categoria_Tienda'  : df_historial['category'],
    'Tipo_de_Movimiento': df_historial['movement_type'],
    'Cantidad'          : df_historial['quantity'],
    'Precio_Unitario'   : df_historial['price_per_unit'],
    'Stock_Minimo'      : df_historial['minimum_stock'],
    'Stock_Restante'    : df_historial['stock_after']
}



datacopy = data.copy()
df = pd.DataFrame(datacopy)

# ---------------------------------------------------------
# PASO CRÍTICO: Calcular el flujo 
# ---------------------------------------------------------

# 1. Ordenamos por fecha para que el cálculo cronológico sea correcto
df = df.sort_values(by=['Producto_Tienda', 'Fecha_Hora'])

# 2. Creamos una columna temporal 'Cambio_Stock'
df['Cambio_Stock'] = np.where(
    df['Tipo_de_Movimiento'] == 'Salida', 
    df['Cantidad'] * -1, 
    df['Cantidad']
)

# 3. Calculamos el Stock Acumulado (Running Total) por producto
df['Stock_Calculado'] = df.groupby('Producto_Tienda')['Cambio_Stock'].cumsum()

# ---------------------------------------------------------

# 4. Calcular métricas finales
df['Total_Venta'] = df['Cantidad'] * df['Precio_Unitario']
df['Valor_Inventario'] = df['Stock_Calculado'] * df['Precio_Unitario']

# 5. Limpieza (opcional): quitamos la columna auxiliar
# df = df.drop(columns=['Cambio_Stock'])

print(df[['Fecha_Hora', 'Producto_Tienda', 'Tipo_de_Movimiento', 'Cantidad', 'Stock_Calculado']].head(10))


ventas = df.groupby('Producto_Tienda').agg({'Cantidad':'sum', 'Total_Venta':'sum'}).reset_index()
ventas = ventas.sort_values(by='Total_Venta', ascending=True)

fig1 = px.bar(ventas,
              x='Total_Venta',
              y='Producto_Tienda',
              orientation='h',
              title ='Ranking de Ventas por Producto',
              text='Cantidad',
              color_continuous_scale='Viridis'
              )

fig1.update_layout(xaxis_title='Total de Ventas',
                   yaxis_title='Producto',
                   template='plotly_white')
#fig1.show()

df['Estado_Stock'] = np.where(np.array(df['Stock_Calculado']) < np.array(df['Stock_Minimo']), 'Bajo', 'Adecuado')

colores = {'Bajo' :'red','Adecuado':'green'}
##############################################
# Gráfico de barras con alerta de stock
################################################


fig2 = px.bar(df, 
              x='Producto_Tienda', 
              y='Stock_Calculado', 
              color='Estado_Stock',
              title='Alerta de Stock (Rojo = Debajo del Mínimo)',
              color_discrete_map=colores,
              text='Stock_Calculado')

# Agregamos una línea horizontal para marcar el umbral (ejemplo visual)
fig2.add_shape(type="line", x0=-0.5, x1=5.5, y0=5, y1=5, 
               line=dict(color="Black", width=2, dash="dot"))
fig2.update_layout(yaxis_title="Unidades en Bodega")
#fig2.show()

# --- GRÁFICA 3: MAPA DE CALOR (La que faltaba) ---
# Preparamos los datos
df['Dia_Semana'] = df['Fecha_Hora'].dt.day_name()
df['Hora'] = df['Fecha_Hora'].dt.hour
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Agrupamos
actividad = df.groupby(['Dia_Semana', 'Hora']).size().reset_index(name='Movimientos')

fig3 = px.density_heatmap(actividad, 
                          x='Dia_Semana', 
                          y='Hora', 
                          z='Movimientos', 
                          title='Mapa de Calor: Intensidad de Actividad',
                          category_orders={'Dia_Semana': dias_orden},
                          color_continuous_scale='Magma',
                          nbinsy=24) # Para que las horas se vean bien

# Ajuste visual eje Y
fig3.update_layout(yaxis=dict(tickmode='linear', dtick=1, range=[7, 19]))
#fig3.show()



           Fecha_Hora Producto_Tienda Tipo_de_Movimiento  Cantidad  \
1 2025-12-10 15:23:00           apple            Ingreso        50   
2 2025-12-10 15:34:00           apple             Salida        10   
3 2025-12-10 16:10:00           apple            Ingreso         6   
4 2025-12-10 16:11:00           apple            Ingreso         9   
5 2025-12-10 16:19:00           apple             Salida         6   
0 2023-10-01 10:00:00      cell phone            Ingreso         2   
6 2025-12-10 18:04:00      cell phone            Ingreso        10   

   Stock_Calculado  
1               50  
2               40  
3               46  
4               55  
5               49  
0                2  
6               12  


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('Estado_Stock=Adecuado<br>Produ' ... 'lculado=%{text}<extra></extra>'),
              'legendgroup': 'Adecuado',
              'marker': {'color': 'green', 'pattern': {'shape': ''}},
              'name': 'Adecuado',
              'orientation': 'v',
              'showlegend': True,
              'text': {'bdata': 'AAAAAAAASUAAAAAAAABEQAAAAAAAAEdAAAAAAACAS0AAAAAAAIBIQAAAAAAAAChA', 'dtype': 'f8'},
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['apple', 'apple', 'apple', 'apple', 'apple', 'cell phone'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'MiguNzEM', 'dtype': 'i1'},
              'yaxis': 'y'},
             {'hovertemplate': ('Estado_Stock=Bajo<br>Producto_' ... 'lculado=%{text}<extra></extra>'),
              'legendgroup': 'Bajo',
              'marker': {'color': 'red', 'pattern': {'shape': ''}},
              'name': 'Bajo',
              'orientation': 'v',
              'showlegend': True,
              'text': {'bdata': 'AAAAAAAAAEA=', 'dtype': 'f8'},
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['cell phone'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'Ag==', 'dtype': 'i1'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'Estado_Stock'}, 'tracegroupgap': 0},
               'shapes': [{'line': {'color': 'Black', 'dash': 'dot', 'width': 2},
                           'type': 'line',
                           'x0': -0.5,
                           'x1': 5.5,
                           'y0': 5,
                           'y1': 5}],
               'template': '...',
               'title': {'text': 'Alerta de Stock (Rojo = Debajo del Mínimo)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Producto_Tienda'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Unidades en Bodega'}}}
})

Grafica de Ventas

Grafica de Stock

Grafico de Calor para la Actividad

In [59]:
# --- GRÁFICA 4: CUADRANTE DE EFICIENCIA (Stock vs Ventas) ---

# 1. Preparamos los datos: Stock Actual vs Total Vendido
df_scatter = df.groupby('Producto_Tienda').agg({
    'Total_Venta': 'sum',          # Eje Y: Qué tanto se mueve el dinero 
    'Stock_Calculado': 'last',     # Eje X: Cuánto tengo 
    'Categoria_Tienda': 'first'    # Para colorear por categoría
}).reset_index()

# 2. Creamos el gráfico
fig4 = px.scatter(df_scatter, 
                  x='Stock_Calculado', 
                  y='Total_Venta',
                  color='Categoria_Tienda', # Colores por categoría
                  size='Total_Venta',       # El tamaño de la burbuja es la venta
                  hover_name='Producto_Tienda',
                  text='Producto_Tienda',
                  title='<b>Cuadrante de Eficiencia:</b> ¿Qué sobra y qué falta?',
                  template='plotly_white')

# 3. Agregamos líneas promedio para dividir en 4 cuadrantes
promedio_ventas = df_scatter['Total_Venta'].mean()
promedio_stock = df_scatter['Stock_Calculado'].mean()

fig4.add_vline(x=promedio_stock, line_dash="dot", annotation_text="Stock Promedio")
fig4.add_hline(y=promedio_ventas, line_dash="dot", annotation_text="Ventas Promedio")

# Etiquetas de los cuadrantes (Opcional, para que se entienda mejor)
# Cuadrante: Vende Mucho / Poco Stock (Peligro)
fig4.add_annotation(x=promedio_stock*0.5, y=promedio_ventas*1.5, 
                    text=" PELIGRO (Reponer)", showarrow=False, font=dict(color="red"))

# Cuadrante: Vende Poco / Mucho Stock (Exceso)
fig4.add_annotation(x=promedio_stock*1.5, y=promedio_ventas*0.5, 
                    text=" EXCESO (Ofertar)", showarrow=False, font=dict(color="gray"))

#fig4.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>Cat' ... '_Tienda=%{text}<extra></extra>'),
              'hovertext': array(['apple'], dtype=object),
              'legendgroup': 'fruits',
              'marker': {'color': '#636efa',
                         'size': {'bdata': 'AAAAAABQeUA=', 'dtype': 'f8'},
                         'sizemode': 'area',
                         'sizeref': 30.0,
                         'symbol': 'circle'},
              'mode': 'markers+text',
              'name': 'fruits',
              'orientation': 'v',
              'showlegend': True,
              'text': array(['apple'], dtype=object),
              'type': 'scatter',
              'x': {'bdata': 'MQ==', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABQeUA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('<b>%{hovertext}</b><br><br>Cat' ... '_Tienda=%{text}<extra></extra>'),
              'hovertext': array(['cell phone'], dtype=object),
              'legendgroup': 'Electronics',
              'marker': {'color': '#EF553B',
                         'size': {'bdata': 'AAAAAABwx0A=', 'dtype': 'f8'},
                         'sizemode': 'area',
                         'sizeref': 30.0,
                         'symbol': 'circle'},
              'mode': 'markers+text',
              'name': 'Electronics',
              'orientation': 'v',
              'showlegend': True,
              'text': array(['cell phone'], dtype=object),
              'type': 'scatter',
              'x': {'bdata': 'DA==', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABwx0A=', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'Stock Promedio',
                                'x': 30.5,
                                'xanchor': 'left',
                                'xref': 'x',
                                'y': 1,
                                'yanchor': 'top',
                                'yref': 'y domain'},
                               {'showarrow': False,
                                'text': 'Ventas Promedio',
                                'x': 1,
                                'xanchor': 'right',
                                'xref': 'x domain',
                                'y': 6202.5,
                                'yanchor': 'bottom',
                                'yref': 'y'},
                               {'font': {'color': 'red'},
                                'showarrow': False,
                                'text': ' PELIGRO (Reponer)',
                                'x': 15.25,
                                'y': 9303.75},
                               {'font': {'color': 'gray'},
                                'showarrow': False,
                                'text': ' EXCESO (Ofertar)',
                                'x': 45.75,
                                'y': 3101.25}],
               'legend': {'itemsizing': 'constant', 'title': {'text': 'Categoria_Tienda'}, 'tracegroupgap': 0},
               'shapes': [{'line': {'dash': 'dot'},
                           'type': 'line',
                           'x0': 30.5,
                           'x1': 30.5,
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'},
                          {'line': {'dash': 'dot'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 6202.5,
                           'y1': 6202.5,
                           'yref': 'y'}],
               'template': '...',
               'title': {'text': '<b>Cuadrante de Eficiencia:</b> ¿Qué sobra y qué falta?'},
               'xaxis': {'anchor': 'y', 

In [69]:
# Muestra cuánto dinero hay en cada Categoría y Producto
fig5 = px.sunburst(df, 
                   path=['Categoria_Tienda', 'Producto_Tienda'], 
                   values='Valor_Inventario',
                   color='Valor_Inventario',
                   title='<b>Distribución del Capital:</b> ¿Dónde está mi dinero?',
                   color_continuous_scale='RdBu')
fig5.update_layout(yaxis=dict(tickmode='linear', dtick=1, range=[7, 19]))
#fig5.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'branchvalues': 'total',
              'customdata': {'bdata': 'VVVVVVVRbkDbtm3btqXEQFVVVVVVUW5A27Zt27alxEA=', 'dtype': 'f8', 'shape': '4, 1'},
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('labels=%{label}<br>Valor_Inven' ... 'ntario=%{color}<extra></extra>'),
              'ids': array(['fruits/apple', 'Electronics/cell phone', 'fruits', 'Electronics'],
                           dtype=object),
              'labels': array(['apple', 'cell phone', 'fruits', 'Electronics'], dtype=object),
              'marker': {'coloraxis': 'coloraxis',
                         'colors': {'bdata': 'VVVVVVVRbkDbtm3btqXEQFVVVVVVUW5A27Zt27alxEA=', 'dtype': 'f8'}},
              'name': '',
              'parents': array(['fruits', 'Electronics', '', ''], dtype=object),
              'type': 'sunburst',
              'values': {'bdata': 'AAAAAADAkkAAAAAAAFjLQAAAAAAAwJJAAAAAAABYy0A=', 'dtype': 'f8'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Valor_Inventario'}},
                             'colorscale': [[0.0, 'rgb(103,0,31)'], [0.1,
                                            'rgb(178,24,43)'], [0.2,
                                            'rgb(214,96,77)'], [0.3,
                                            'rgb(244,165,130)'], [0.4,
                                            'rgb(253,219,199)'], [0.5,
                                            'rgb(247,247,247)'], [0.6,
                                            'rgb(209,229,240)'], [0.7,
                                            'rgb(146,197,222)'], [0.8,
                                            'rgb(67,147,195)'], [0.9,
                                            'rgb(33,102,172)'], [1.0,
                                            'rgb(5,48,97)']]},
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': '<b>Distribución del Capital:</b> ¿Dónde está mi dinero?'},
               'yaxis': {'dtick': 1, 'range': [7, 19], 'tickmode': 'linear'}}
})

In [73]:
# Calculamos la contribución de cada producto a las ventas totales
df_abc = df.groupby('Producto_Tienda')['Total_Venta'].sum().reset_index()
df_abc = df_abc.sort_values(by='Total_Venta', ascending=False)

# Calculamos porcentaje acumulado
df_abc['Porcentaje_Acumulado'] = (df_abc['Total_Venta'].cumsum() / df_abc['Total_Venta'].sum()) * 100

# Creamos el gráfico combinado (Barras + Línea)
fig6 = go.Figure()

# Barras: Venta por producto
fig6.add_trace(go.Bar(
    x=df_abc['Producto_Tienda'], 
    y=df_abc['Total_Venta'],
    name='Venta ($)',
    marker_color='rgb(55, 83, 109)'
))

# Línea: Porcentaje acumulado
fig6.add_trace(go.Scatter(
    x=df_abc['Producto_Tienda'], 
    y=df_abc['Porcentaje_Acumulado'],
    name='% Acumulado',
    yaxis='y2',
    marker_color='rgb(255, 65, 54)'
))

fig6.update_layout(
    title='<b>Análisis ABC (Pareto):</b> El 80% de tus ingresos viene de estos productos',
    yaxis_title='Ventas ($)',
    yaxis2=dict(
        title='Porcentaje Acumulado (%)',
        overlaying='y',
        side='right',
        range=[0, 110]
    ),
    template='plotly_white'
)

# Línea del 80% (Regla de Pareto)
fig6.add_hline(y=80, line_dash="dash", line_color="green", annotation_text="Límite 80% (Clase A)", yref="y2")
#fig6.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': 'rgb(55, 83, 109)'},
              'name': 'Venta ($)',
              'type': 'bar',
              'x': array(['cell phone', 'apple'], dtype=object),
              'y': {'bdata': 'AAAAAABwx0AAAAAAAFB5QA==', 'dtype': 'f8'}},
             {'marker': {'color': 'rgb(255, 65, 54)'},
              'name': '% Acumulado',
              'type': 'scatter',
              'x': array(['cell phone', 'apple'], dtype=object),
              'y': {'bdata': 'rBWOTw0vWEAAAAAAAABZQA==', 'dtype': 'f8'},
              'yaxis': 'y2'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'Límite 80% (Clase A)',
                                'x': 1,
                                'xanchor': 'right',
                                'xref': 'x domain',
                                'y': 80,
                                'yanchor': 'bottom',
                                'yref': 'y2'}],
               'shapes': [{'line': {'color': 'green', 'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 80,
                           'y1': 80,
                           'yref': 'y2'}],
               'template': '...',
               'title': {'text': '<b>Análisis ABC (Pareto):</b> El 80% de tus ingresos viene de estos productos'},
               'yaxis': {'title': {'text': 'Ventas ($)'}},
               'yaxis2': {'overlaying': 'y',
                          'range': [0, 110],
                          'side': 'right',
                          'title': {'text': 'Porcentaje Acumulado (%)'}}}
})

In [74]:
# Calculamos qué porcentaje de productos tienen stock BAJO
total_productos = len(df)
productos_bajos = len(df[df['Estado_Stock'] == 'Bajo'])
porcentaje_riesgo = (productos_bajos / total_productos) * 100

fig7 = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = porcentaje_riesgo,
    title = {'text': "<b>Nivel de Riesgo de Stock</b><br>(% Productos Agotándose)"},
    gauge = {
        'axis': {'range': [0, 100]},
        'bar': {'color': "darkblue"},
        'steps': [
            {'range': [0, 30], 'color': "green"},  # 0-30% Riesgo bajo (Bien)
            {'range': [30, 70], 'color': "yellow"}, # 30-70% Riesgo medio
            {'range': [70, 100], 'color': "red"}   # 70-100% Riesgo alto (Mal)
        ],
        'threshold': {
            'line': {'color': "black", 'width': 4},
            'thickness': 0.75,
            'value': porcentaje_riesgo
        }
    }
))
fig7.add_hline(y=80, line_dash="dash", line_color="green", annotation_text="KPI DE SALUD DE INVENTARIO", yref="y2")

#fig7.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'gauge': {'axis': {'range': [0, 100]},
                        'bar': {'color': 'darkblue'},
                        'steps': [{'color': 'green', 'range': [0, 30]},
                                  {'color': 'yellow', 'range': [30, 70]},
                                  {'color': 'red', 'range': [70, 100]}],
                        'threshold': {'line': {'color': 'black', 'width': 4},
                                      'thickness': 0.75,
                                      'value': 14.285714285714285}},
              'mode': 'gauge+number',
              'title': {'text': '<b>Nivel de Riesgo de Stock</b><br>(% Productos Agotándose)'},
              'type': 'indicator',
              'value': 14.285714285714285}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'KPI DE SALUD DE INVENTARIO',
                                'x': 1,
                                'xanchor': 'right',
                                'xref': 'x domain',
                                'y': 80,
                                'yanchor': 'bottom',
                                'yref': 'y2'}],
               'shapes': [{'line': {'color': 'green', 'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 80,
                           'y1': 80,
                           'yref': 'y2'}],
               'template': '...'}
})